# 交叉熵
在信息论中，交叉熵衡量的是当你有一个估计的概率分布 q，并用这个分布来编码信息时，
平均需要多少位（比特）来正确地识别一个事件。简而言之，交叉熵描述了使用非最佳概率分布进行编码时，传达事件所需的额外信息量。交叉熵函数和负对数似然形式等价。
由于y是一个 长度为q的独热编码向量，所以除了一个项以外的所有项j都消失了。由于所有ˆ yj 都是预测的概率，所以它们的对数永远不会大于0 <br>
补充，熵的公式、相对熵（KL散度）公式，以及它们与交叉熵的关系。<br>
熵的公式(熵就是平均信息量,信息量的公式为$I(x) = - \log p(x)$)：<br>
$H(p) = - \sum_{i=1}^{N} p(x_i) \log q(x_i)$ <br>
相对熵（KL散度）公式：<br>
$D_{KL}(P\| Q) = \sum_{i=1}^{N} p(x_i) \log\left(\frac{p(x_i)}{q(x_i)}\right)$<br>
熵、交叉熵和KL散度的关系——交叉熵可以分解为熵和KL散度的和：<br>
$H(P,Q) = H(P)+D_{KL}(P\| Q)$<br>
交叉熵的公式：<br>
$H(P, Q) = - \sum_{i=1}^{N} p(x_i) \log p(x_i)$ <br>
其中：<br>
- $p(x_i)$是真实分布P中事件$x_i$发生的概率<br>
- $q(x_i)$是我们用近似分布Q对事件$x_i$的预测概率。<br>

这个公式表示，在真实分布下下事件发生的频率为$p(x_i)$时，我们用分布Q来编码这些事件所需的平均信息量。直观地说，如果我们一直用Q来编码或预测P中的事件，我们每次会“付出”多少信息代价(“代价”指的是我们需要消耗多少比特或多少单位的信息来正确地编码或预测某个事件。)。<br>
简化为分类问题中使用独热编码的形式<br>
$H(P, Q) = - \sum_{i=1}^{N} y_i \log \hat{y}_i$
